In [1]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_924decc15eed45238b5eb7d3b7cc7c26"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_8a7e45d110c24152980c1aa2587b010c"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_60ecea6cbfae47f2954238c219c8f2d2"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_4d0f0b1a707f4e2392233f916158897a"},{"name":"output2","type":0,"uri":"tmp_3ec9d1e17dad4021b6110e8c75299812"},{"name":"output3","type":0,"uri":"tmp_e3790caf415a4a248fb11cd7667153e6"},{"name":"output4","type":0,"uri":"tmp_055ddd8c9be54f40ada7a0a081ef9c1d"}]'

# 自定义参数


In [2]:
!pip install torch
!pip install ipdb
import pandas as pd
import numpy as np
import re
import random
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.externals import joblib
import joblib
import wfio
from ustciscrBDL_B import get_score
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
import pickle
#!pip install ipdb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [3]:
#!rm model.pth

In [4]:
#!wget home.ustc.edu.cn/~zrj1997/model.pth

In [5]:
#!wget -c --referer=https://pan.baidu.com/s/1jRVF5SdlTqyByZ3OryDBHw -O model.pth "https://bjbgp01.baidupcs.com/file/76524301enb25e7340f3ec0032fa6a79?bkt=en-43ea5360a23c0e2022955787247f6b8b05228903fb8d8985d9645be186dc59a55cf42c99b5df6e50cbc57aede2281f222ab6768ec6fe2d205137ca032d50407f&fid=3734286207-250528-727342125319077&time=1576912347&sign=FDTAXGERLQBHSKfW-DCb740ccc5511e5e8fedcff06b081203-wpBBlce0AVhuL2NL3c6JWhqqorY%3D&to=75&size=16354&sta_dx=16354&sta_cs=2&sta_ft=pth&sta_ct=1&sta_mt=1&fm2=MH%2CYangquan%2CAnywhere%2C%2Canhui%2Cce&ctime=1576742908&mtime=1576742908&resv0=cdnback&resv1=0&resv2=rlim&resv3=5&resv4=16354&vuk=3734286207&iv=0&htype=&randtype=&newver=1&newfm=1&secfm=1&flow_ver=3&pkey=en-87e26db05518e6cb7a9c5c38a0615fa079d42aca88523e757752a51d2c4146a966ee4195b1b8f2eebbc2c0005fb1bd0736f384daeb653c1e305a5e1275657320&sl=68616270&expires=8h&rt=sh&r=553265769&vbdid=4269441613&fin=model.pth&fn=model.pth&rtype=1&dp-logid=8224955292230391832&dp-callid=0.1&hps=1&tsl=200&csl=200&csign=Ms6794UnRgvx8TBYvFZ4bj4bJYE%3D&so=0&ut=6&uter=4&serv=0&uc=2825019094&ti=cdac69781712398031a67f7aeda69655bb55fe4fb6f7c28d305a5e1275657320&reqlabel=250528_f_9edf4b80b1193f7e8742fcf0c8451f9a_-1_1fad5c55b4c28a72c2a489d697dc94a3&by=themis"

In [6]:
#读取测试集数据
def get_test_data():
    test_data = wfio.read_dataframe(_INPUT3)
    #删除第一行中文
    #test_data = test_data.drop(0,axis=0,inplace=False)
    return test_data

In [7]:
#读取训练集数据
def get_train_data():
    train_data = wfio.read_dataframe(_INPUT2)
    #删除第一行中文
    #print(train_data.columns.values)
    #train_data = train_data.drop(0,axis=0,inplace=False)
    return train_data

In [8]:
def Digital(d):
    tmp1 = np.array(d)
    tmp2 = []
    for x in tmp1:
        if x.isdigit() or x.split(".")[0].isdigit():
            tmp2.append(str(x))
        else:
            tmp2.append("999999")
    return np.array(tmp2)

In [9]:
def FP_Q_feature(data, FP_Q):
    years = ['2017', '2018', '2019']
    seasons = ['q1', 'q2', 'q3', 'q4']
    names = ['fphdsl', 'kps', 'kpje', 'kpsl', 'kpse', 'jxje', 'jxsl', 'rkse']
    #names = ['kpje', 'kpsl', 'kpse', 'jxje', 'jxsl', 'rkse']
    KPJE_mark = np.zeros((len(data), len(years)*len(seasons)))
    #import ipdb;ipdb.set_trace()
    i = 0
    for y in years:
        for s in seasons:
            pre = y+s
            #发票增量, 发票顶额
            #print(y, s)
            #import ipdb;ipdb.set_trace()
            for n in range(len(names)):
                data[pre+names[n]] = Digital(data[pre+names[n]])
                FP_Q[pre+names[n]] = data[pre+names[n]]
            FPZL = np.array(data[pre+names[1]].astype('float')) - np.array(data[pre+names[0]].astype('float'))
            FPDE = np.array(data[pre+names[2]].astype('float')) / (np.array(data[pre+names[1]].astype('float'))+1e-4)
            #销项进项比值
            KP_JX = np.array(data[pre+names[2]].astype('float')) / (np.array(data[pre+names[5]].astype('float'))+1e-4)
            #计算的开票税额，进项税额，入库税额
            KPSE = np.array(data[pre+names[2]].astype('float')) * np.array(data[pre+names[3]].astype('float'))
            JXSE = np.array(data[pre+names[5]].astype('float')) * np.array(data[pre+names[6]].astype('float'))
            RKSE = np.array(np.where(data[pre+names[6]].astype('float')==0.03, KPSE, KPSE-JXSE))
            #与真实的开票税额、入库税额之间的差值
            KPSE_diff = KPSE - data[pre+names[4]].astype('float')
            RKSE_diff = RKSE - data[pre+names[7]].astype('float')
            
            #FP_Q[pre+'FPZL'] = FPZL
            #FP_Q[pre+'FPDE'] = FPDE
            FP_Q[pre+'KP_JX'] = KP_JX
            FP_Q[pre+'KPSE'] = KPSE
            FP_Q[pre+'JXSE'] = JXSE
            FP_Q[pre+'RKSE'] = RKSE
            #FP_Q[pre+'KPSE_diff'] = KPSE_diff
            #FP_Q[pre+"RKSE_diff"] = RKSE_diff

            KPJE_mark[:, i] = data[pre+names[2]].astype('float')
            i += 1
    tmp_CXSJ = []
    for i in range(len(data)):
        tmp = np.where(KPJE_mark[i,:]!=0)[0]
        if len(tmp)!=0:
            tmp_CXSJ.extend([tmp[-1]-tmp[0]])
        else:
            tmp_CXSJ.extend([0])
    #FP_Q['CXSJ'] = tmp_CXSJ
    return

In [10]:
def extract_feature(data):
    feature = {}
    feature['NSRSBH'] = data['zjnsrsbh']

    # WXHY:是否属于危险行业: 0->dangerous
    #col_data = data['HYML']
    #dangerous = set(['批发和零售业','建筑业','商贸','批发','零售业','零售','服务业','服务','咨询','鉴证','人力资源', '纺织','服饰','木材加工','中药','农产品','收购','租赁'])
    #tmp_feature = []
    #for f in col_data:
    #    words = set(jieba.lcut(f))
    #    tmp_feature.extend([not words & dangerous])
    #feature['WXHY'] = tmp_feature

    # ZCDZ_repeat/SCJYDZ_repeat：地址是否重复出现: 0->yes
    #ZCDZ = data['ZCDZ']
    #SCJYDZ = data['SCJYDZ']
    #ZCDZ = address_repeat_feature(ZCDZ)
    #SCJYDZ = address_repeat_feature(SCJYDZ)
    #feature['ZCDZ_repeat'] = ZCDZ
    #feature['SCJYDZ_repeat'] = SCJYDZ

    # 地址是否非商业区:0-?非商业区
    #ZCDZ = data['ZCDZ']
    #SCJYDZ = data['SCJYDZ']
    #ZCDZ = address_nobusiness_feature(ZCDZ)
    #SCJYDZ = address_nobusiness_feature(SCJYDZ)

    # 多户企业法人/财务人员相同/身份交叉:0->相同
    #FDDBRXM = data['FDDBRXM']
    #BSRXM = data['BSRXM']
    #identity = identity_feature(FDDBRXM, BSRXM)

    # 季节发票特征
    FP_Q_feature(data, feature) 

    #登记到领票之间的间隔
    #standard = ['2017/01/01']*len(data)
    #diff = pd.to_datetime(standard) - pd.to_datetime(data['djrq']) + pd.Timedelta(days = 730)
    #feature['LPJG'] = np.array(diff)

    return feature

In [11]:
def generate_feature_data(features):
    feature_data = []
    keys = features.keys()
    for key in keys:
        if key != 'NSRSBH':
            feature_data.append(features[key])
    return np.array(feature_data).T, features['NSRSBH']
def generate_train_data(train_feauture_data,train_data):
    train_y = np.array(train_data['yc']).astype('float')
    #import ipdb;ipdb.set_trace()
    #train_feature_data  = [x.astype('float') for x in train_feature_data]
    pos_len = len(np.where(train_y>=0.5)[0])
    neg_idx = np.where(train_y<0.5)[0]
    neg_idx_sample = random.sample(list(neg_idx), int(pos_len*3/2))
    pos_idx = np.where(train_y>=0.5)[0]
    idx = np.concatenate([pos_idx, neg_idx_sample], 0)
    return train_feature_data[idx], train_y[idx]
def generate_test_data(feature_data):
    
    #NSRSBH = list(NSRSBH)
    #for id in list(train_data['zjnsrsbh']):
    #    sub = NSRSBH.index(id)
    #    train_x.append(feature_data[sub])
    return feature_data

In [12]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.batchnorm = nn.Sequential(
            nn.BatchNorm1d(input_size),
        )
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 4, 3),
            )
        self.batchnorm_conv = nn.Sequential(
            nn.BatchNorm1d(4),
        )
        self.conv2 = nn.Conv1d(4, 4, 3)
        self.linear = nn.Sequential(
            nn.Linear(4*140, 4),
            #nn.ReLU(True),
            #nn.Conv1d(16, 16, 16),
            nn.Linear(4, 1),
            nn.Sigmoid(),
            )
    def forward(self, feature_data):
        #import ipdb;ipdb.set_trace()
        norm_output = self.batchnorm(feature_data)
        norm_output = norm_output.view(norm_output.size(0), 1, norm_output.size(1))
        conv_output = self.conv1(norm_output)
        conv_output = self.batchnorm_conv(conv_output)
        conv_output = self.conv2(conv_output)
        conv_output = conv_output.view(conv_output.size(0), -1)
        out = self.linear(conv_output)
        return out
    
class Net2(nn.Module):
    def __init__(self, input_size):
        super(Net2, self).__init__()
        self.batchnorm = nn.Sequential(
            nn.BatchNorm1d(input_size),
        )
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 4, 3),
            #nn.Conv1d(4, 8, 3),
            )
        self.conv2= nn.Sequential(
            nn.BatchNorm1d(4),
            nn.Conv1d(4, 4, 3)
        )
        self.linear = nn.Sequential(
            nn.Linear(4*140, 4),
            #nn.ReLU(True),
            #nn.Conv1d(16, 4, 3),
            #nn.Linear(32, 16),
            #nn.Linear(16, 4),
            nn.Linear(4, 1),
            nn.Sigmoid(),
            )
    def forward(self, feature_data):
        #import ipdb;ipdb.set_trace()
        #norm_output = self.batchnorm(feature_data)
        norm_output = norm_output.view(norm_output.size(0), 1, norm_output.size(1))
        conv_output_1 = self.conv1(norm_output)

        conv_output_2 = self.conv2(conv_output_1)
        #conv_output_2 = self.batch2(conv_output_1)
        pool_output = conv_output_2.view(conv_output_2.size(0), -1)
        out = self.linear(pool_output)
        return out

class FP_dataset(data.Dataset):
    def __init__(self, x_data, y_data):
        super(FP_dataset, self).__init__()
        self.x = x_data
        self.y = y_data
    def __getitem__(self, index):
        #import ipdb;ipdb.set_trace()
        return self.x[index].astype(np.float32), self.y[index]
    def __len__(self):
        return len(self.x)

In [13]:
def create_verify_data(NSRSBH, predictions):
    predictions = predictions.reshape(len(predictions))
    predictions = np.where(predictions>0.60, 1, predictions)
    predictions = np.where(predictions<0.13, 0, predictions)
    prob = {'zjnsrsbh':NSRSBH, 'Probability':predictions}
    user_verify_data = pd.DataFrame(prob)

    return user_verify_data

In [14]:
#train_data = get_train_data()
test_data = get_test_data()
get_score.post_user_id('18')

In [15]:
#train_features = extract_feature(train_data) 
test_features = extract_feature(test_data)

In [16]:
test_feature_data, NSRSBH=generate_feature_data(test_features)
test_x = generate_test_data(test_feature_data)
net = Net(len(test_feature_data[0]))
#train_x = np.array([x.astype(np.float32) for x in train_x])

#fp_dataset = FP_dataset(train_x, train_y)
#loader = DataLoader(dataset = fp_dataset, batch_size = 128, shuffle = True)
#opt = torch.optim.Adam(net.parameters())
#loss_func = torch.nn.BCELoss()
#net.train()
#for epoch in range(50):
#    print('Epoch: ', epoch, end = ' ')
#    loss_array = []
#    #import ipdb;ipdb.set_trace()
#    for step, (x, y) in enumerate(loader):
#        output = net(x)
#        #import ipdb;ipdb.set_trace()
#        loss = loss_func(output.squeeze(), y)
#        opt.zero_grad()
#        loss.backward()
#        opt.step()
#        loss_array.append(loss.data.numpy())
#    print('loss: ', loss.data.numpy())
#torch.save(net, 'test.pth')
##net = torch.load('model.pkl')


In [17]:
!ls 

model.pth  model.pth.1	model.pth.2


In [18]:
net = torch.load('model.pth')

In [19]:
net.eval()
#import ipdb;ipdb.set_trace()
feature_data = np.array([x.astype(np.float32) for x in test_x])
feature_data = torch.from_numpy(feature_data)
predictions = net(feature_data).detach().numpy()

In [20]:
user_verify_data = create_verify_data(test_data['zjnsrsbh'], predictions)

In [21]:
get_score.post_verify_data(user_verify_data)

User-id:18
程序开始运行时间为：2019-12-22 14:44:29.062054
程序结束运行时间为：2019-12-22 14:44:30.545579
程序运行时间（去除打分耗时）为：1.483525
程序的准确率为：95.60628525
最终得分为：96.04565673
